# Increasing the Maximum Message Size for gRPC


## Running this notebook

You will need to start Jupyter with settings to allow for large payloads, for example:

```
jupyter notebook --NotebookApp.iopub_data_rate_limit=1000000000
```

## Setup Seldon Core

Use the setup notebook to [Setup Cluster](seldon_core_setup.ipynb#Setup-Cluster) with [Ambassador Ingress](seldon_core_setup.ipynb#Ambassador) and [Install Seldon Core](seldon_core_setup.ipynb#Install-Seldon-Core). Instructions [also online](./seldon_core_setup.html).

In [ ]:
!kubectl create namespace seldon

In [ ]:
!kubectl config set-context $(kubectl config current-context) --namespace=seldon

In [ ]:
!pygmentize resources/model_long_timeouts.json

## Create Seldon Deployment

Deploy the runtime graph to kubernetes.

In [ ]:
!kubectl apply -f resources/model_long_timeouts.json -n seldon

In [ ]:
!kubectl rollout status deploy/model-long-timeout-test-0

## Get predictions

In [4]:
from seldon_core.seldon_client import SeldonClient
sc = SeldonClient(deployment_name="model-long-timeout",namespace="seldon", 
                  grpc_max_send_message_length=50 * 1024 * 1024, grpc_max_receive_message_length=50 * 1024 * 1024)

/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/clive/anaconda3/envs/seldon-core/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/clive/anaconda3/envs/seldon-

Send a small request which should suceed.

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc")
assert(r.success==True)
print(r)

Send a large request which will fail as the default for the model will be 4G.

In [ ]:
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1000000,1))
print(r.success,r.msg)

In [ ]:
!kubectl delete -f resources/model_long_timeouts.json

## Allowing larger gRPC messages

Now we change our SeldonDeployment to include a annotation for max grpx message size.

In [7]:
!pygmentize resources/model_grpc_size.json

{
    "apiVersion": "machinelearning.seldon.io/v1alpha2",
    "kind": "SeldonDeployment",
    "metadata": {
        "labels": {
            "app": "seldon"
        },
        "name": "seldon-model"
    },
    "spec": {
        "annotations": {
	    "seldon.io/grpc-max-message-size":"10000000",
	    "seldon.io/rest-timeout":"100000",
	    "seldon.io/grpc-timeout":"100000"
        },
        "name": "test-deployment",
        "predictors": [
            {
                "componentSpecs": [{
                    "spec": {
                        "containers": [
                            {
                                "image": "seldonio/mock_classifier_grpc:1.5",
                                "imagePullPolicy": "IfNotPresent",
                                "name": "classifier",
                                "resources": {
                                    "requests": {
                                        "memory": "1Mi"
                                    }
               

In [19]:
!kubectl create -f resources/model_grpc_size.json -n seldon

seldondeployment.machinelearning.seldon.io/seldon-model created


In [20]:
!kubectl rollout status deploy/seldon-model-grpc-size-0 

Waiting for deployment "seldon-model-grpc-size-0" rollout to finish: 0 of 1 updated replicas are available...
deployment "seldon-model-grpc-size-0" successfully rolled out


Send a request via ambassador. This should succeed.

In [21]:
sc = SeldonClient(deployment_name="seldon-model",namespace="seldon",
                  grpc_max_send_message_length=50 * 1024 * 1024, grpc_max_receive_message_length=50 * 1024 * 1024)
r = sc.predict(gateway="ambassador",transport="grpc",shape=(1000000,1))
assert(r.success==True)
print(r.success)

True


In [22]:
!kubectl delete -f resources/model_grpc_size.json -n seldon

seldondeployment.machinelearning.seldon.io "seldon-model" deleted
